In [1]:
import pandas as pd
from scipy import stats

In [2]:
distances = pd.read_csv("../DataAcquisition/shortest_distances.csv")

Checking if the distances are normally distributed:

In [ ]:
stats.normaltest(distances['distance_km']).pvalue

1.14823205293538e-230

1. Splitting the distances into equal length bins and seeing how many outages occured that have distances in that distance interval for example from (0,3), is the number of outages that had a distance of between 0 to 3 to the nearest station.
2. Now we groupby these intervals and aggregate by counting the number of outages that are in this interval. This gives us outages per distance
3. We can do a statistical test like a T-test to see if there is a significant different between the first half of these outages compared to the second half of these outages.

In [12]:
#this cuts the data into 3 equal width bins.
data = pd.Series([2,19,1,20, 13, 19, 24, 30])
bins = pd.cut(data, bins=3)
print(bins)

0     (0.971, 10.667]
1    (10.667, 20.333]
2     (0.971, 10.667]
3    (10.667, 20.333]
4    (10.667, 20.333]
5    (10.667, 20.333]
6      (20.333, 30.0]
7      (20.333, 30.0]
dtype: category
Categories (3, interval[float64, right]): [(0.971, 10.667] < (10.667, 20.333] < (20.333, 30.0]]


In [31]:
n = 10 #number of bins
distance_bins = pd.cut(distances['distance_km'], bins=n)
distances['distance_bin'] = distance_bins
outages_per_dist = distances.groupby(['distance_bin']).size()
outages_per_dist = pd.DataFrame(outages_per_dist)
outages_per_dist = outages_per_dist.rename(columns={0:"#of outages"})
outages_per_dist


C:\Users\dorsa\AppData\Local\Temp\ipykernel_11200\1079504366.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  outages_per_dist = distances.groupby(['distance_bin']).size()


,#of outages
distance_bin,
"(-0.396, 45.623]",548
"(45.623, 91.186]",2
"(91.186, 136.749]",3
"(136.749, 182.312]",1
"(182.312, 227.875]",1
"(227.875, 273.439]",0
"(273.439, 319.002]",0
"(319.002, 364.565]",0
"(364.565, 410.128]",0
